<a href="https://colab.research.google.com/github/EricCallaway/COSC_5313_CNN_model_HW-5/blob/test/CNN_model_HW_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import necessary Libraries

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import pandas as pd


Import Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
img_data_path_1 = "/content/drive/MyDrive/Data/HW#5_Dataset/imgs1/"
img_data_path_2 = "/content/drive/MyDrive/Data/HW#5_Dataset/imgs2/"
file_data = "/content/drive/MyDrive/Data/HW#5_Dataset/file_data.csv"
class_0_path = "/content/drive/MyDrive/Data/HW#5_Dataset/class_0.csv"
class_1_path = "/content/drive/MyDrive/Data/HW#5_Dataset/class_1.csv"
class_2_path = "/content/drive/MyDrive/Data/HW#5_Dataset/class_2.csv"
class_3_path = "/content/drive/MyDrive/Data/HW#5_Dataset/class_3.csv"


df = pd.read_csv(file_data)
df_class_0 = pd.read_csv(class_0_path)
df_class_1 = pd.read_csv(class_1_path)
df_class_2 = pd.read_csv(class_2_path)
df_class_3 = pd.read_csv(class_3_path)


seq_nums = df['seq_num'].values
file_name = df[' file_name'].values
class_label = df[' class_label'].values



Split each class dataset into train, test, split categories.

In [ ]:
import torchvision
import torch
import torchvision.transforms as transforms
import os
import matplotlib.pyplot as plt
import numpy as np

x_train_0 = df_class_0[' file_name']
y_train_0 = df_class_0[' class_label']


In [ ]:
print(len(df))

4349


This essentially maps the file names to the corresponding class labels one for one. Example, the first file name is 000000 and the class label is 0.

In [ ]:
ds_train = tf.data.Dataset.from_tensor_slices((file_name, class_label))

In [ ]:
def read_img_one(image_file, label):
  image = tf.io.read_file(img_data_path_1 + image_file)
  image = tf.image.decode_image(image, channels=3, dtype=tf.float32)
  return image, label

def read_img_two(image_file, label):
  image = tf.io.read_file(img_data_path_2 + image_file)
  image = tf.image.decode_image(image, channels=3, dtype=tf.float32)
  return image, label

In [ ]:

i, l = read_img_one('000038.bmp', 0)
print(i)
print(l)

In [ ]:
import os
files_1 = sorted(os.listdir(img_data_path_1))
class_0_train_list = []
i = 0
for file in files_1:
  if int(file.strip('.bmp')) == df_class_0[' file_name'].index[i]:
    class_0_train_list.append(read_img_one(str(file), 0))
  i = i + 1
print(len(files_1))
print(len(class_0_train_list))
    

files_2 = sorted(os.listdir(img_data_path_2))
x = 2000
for file in files_2:
  if x == 3450:
    break
  if int(file.strip('.bmp')) == df_class_0[' file_name'].index[x]:
    class_0_train_list.append(read_img_two(str(file), 0))
  x = x + 1
print(len(files_2))
print(len(class_0_train_list))
  



2000
2000
2349
3450


In [ ]:
type(int(df_class_1[' file_name'].values[0]))


int

In [ ]:
import os
files_1 = sorted(os.listdir(img_data_path_1))
class_1_train_list = []
i = 0
for file in files_1:
  if i == 2000 or i == len(df_class_1):
    break
  if int(file.strip('.bmp')) == int(df_class_1[' file_name'].values[i]):
    class_1_train_list.append(read_img_one(str(file), 1))
  i = i + 1
print(len(files_1))
print(len(class_1_train_list))
    

files_2 = sorted(os.listdir(img_data_path_2))
x = 0
for file in files_2:
  if x == 243:
    break
  # print(df_class_1[' file_name'].values[x])
  # print(file.strip('.bmp'))
  if int(file.strip('.bmp')) == int(df_class_1[' file_name'].values[x]):
    class_1_train_list.append(read_img_two(str(file), 1))
  if int(file.strip('.bmp')) >= 3450:
    x = x + 1


  
print(len(files_2))
print(len(class_1_train_list))

2000
0
2349
243


In [ ]:
df_class_2[' file_name'].values

In [ ]:
len(df_class_2)

394

In [ ]:
import os
files_1 = sorted(os.listdir(img_data_path_1))
class_2_train_list = []
i = 0
for file in files_1:
  if i == 2000 or i == len(df_class_2):
    break
  if int(file.strip('.bmp')) == int(df_class_2[' file_name'].values[i]):
    class_2_train_list.append(read_img_one(str(file), 2))
  i = i + 1
print(len(files_1))
print(len(class_2_train_list))
    

files_2 = sorted(os.listdir(img_data_path_2))
x = 0
for file in files_2:
  if x == 394:
    break
  # print(df_class_1[' file_name'].values[x])
  # print(file.strip('.bmp'))
  if int(file.strip('.bmp')) == int(df_class_2[' file_name'].values[x]):
    class_2_train_list.append(read_img_two(str(file), 2))
  if int(file.strip('.bmp')) >= 3693:
    x = x + 1


print('hello')
print(len(files_2))
print(len(class_2_train_list))

In [ ]:
df_class_3[' file_name'].values[0]

4087

In [ ]:
len(df_class_3)

262

In [ ]:
import os
files_1 = sorted(os.listdir(img_data_path_1))
class_3_train_list = []
i = 0
for file in files_1:
  if i == 2000 or i == len(df_class_3):
    break
  if int(file.strip('.bmp')) == int(df_class_3[' file_name'].values[i]):
    class_3_train_list.append(read_img_one(str(file), 3))
  i = i + 1
print(len(files_1))
print(len(class_3_train_list))
    

files_2 = sorted(os.listdir(img_data_path_2))
x = 0
for file in files_2:
  if x == 262:
    break
  # print(df_class_1[' file_name'].values[x])
  # print(file.strip('.bmp'))
  if int(file.strip('.bmp')) == int(df_class_3[' file_name'].values[x]):
    class_3_train_list.append(read_img_two(str(file), 3))
  if int(file.strip('.bmp')) >= 4087:
    x = x + 1


  
print(len(files_2))
print(len(class_3_train_list))

2000
0
2349
262


In [ ]:
from pandas.core.common import random_state
from sklearn.model_selection import train_test_split
x_train_0, x_test_0 = train_test_split(class_0_train_list, test_size=0.15, random_state=42)
x_train_1, x_test_1 = train_test_split(class_1_train_list, test_size=0.15, random_state=42)
x_train_2, x_test_2 = train_test_split(class_2_train_list, test_size=0.15, random_state=42)
x_train_3, x_test_3 = train_test_split(class_3_train_list, test_size=0.15, random_state=42)

In [ ]:
print(len(x_train_0))
print(len(x_test_0))
print(x_train_0[9])

In [ ]:
print(len(x_train_1))
print(len(x_test_1))
print(x_train_1[9])

In [ ]:
print(len(x_train_2))
print(len(x_test_2))
print(x_train_2[9])

In [ ]:
print(len(x_train_3))
print(len(x_test_3))
print(x_train_3[9])